In [1]:
# Matrix Manipulation/Management Libraries
import pandas as pd

# Input/Output Libraries
import os

In [2]:
# Function: Process Activity-By-Contact Model Data 

def read_samples(file: str, sample_info: str):
    cols =  ['Chrom', 'Start', 'End', 'TSS_Chrom', 'TSS_Start', 'TSS_End', 'Gene', 'Score', 'Strand_1', 'Strand_2']
    sample = pd.read_csv(file, sep= '\t', header = None, names = cols)
    sample = sample.drop(['TSS_Chrom', 'TSS_Start', 'TSS_End', 'Score', 'Strand_1', 'Strand_2'], axis = 1) 
    sample['Gene'] = sample['Gene'].str.split('_').str[0]
    sample['Sample'] = sample_info
    
    return sample

In [3]:
# Function: Combine Modules (Enhancers) within n BPs, Maintain Column-Specific (Gene, Sample,etc.) Information

def combine_modules(modules, distance: int, column_header: str):
    modules = modules.sort_values(by=["Start"])
    merged = []
    current_start = modules.iloc[0]["Start"]
    current_end = modules.iloc[0]["End"]
    chromosome = modules.iloc[0]["Chrom"]
    time_point = modules.iloc[0]["Sample"]
    column_information = {time_point: set([modules.iloc[0][column_header]])}

    for i in range(1, len(modules)):
        time_point = modules.iloc[i]["Sample"]
        if modules.iloc[i]["Chrom"] != chromosome:
            merged.append([chromosome, current_start, current_end, column_information])
            current_start = modules.iloc[i]["Start"]
            current_end = modules.iloc[i]["End"]
            chromosome = modules.iloc[i]["Chrom"]
            column_information = {time_point: set([modules.iloc[i][column_header]])}
        elif modules.iloc[i]["Start"] - current_end <= distance:
            current_end = max(modules.iloc[i]["End"], current_end)
            if time_point in column_information:
                column_information[time_point].add(modules.iloc[i][column_header])
            else:
                column_information[time_point] = set([modules.iloc[i][column_header]])
        else:
            merged.append([chromosome, current_start, current_end, column_information])
            current_start = modules.iloc[i]["Start"]
            current_end = modules.iloc[i]["End"]
            column_information = {time_point: set([modules.iloc[i][column_header]])}
    
    merged.append([chromosome, current_start, current_end, column_information]) 
    
    return pd.DataFrame(merged, columns = ["Chrom", "Start", "End", column_header])

In [4]:
# Load Activity-By-Contact Model Samples

file_list = ['0h_sample', '3h_sample', '6h_sample', '12h_sample', '24h_sample', '48h_sample', '72h_sample']
file_dir = '/projectsp/f_ak1833_1/Ziyuan/abc_analysis_organized/further_analysis/'

samples = []

for file_name in file_list: 
    file_path = file_dir + file_name + '.bedpe'
    sample = read_samples(file_path, file_name)
    samples.append(sample)
    
uncollapsed_module_regions = pd.concat(samples)

In [5]:
# Collapse Modules (Enhancers) within 100 BPs
module_regions_genes = combine_modules(uncollapsed_module_regions, 100, "Gene")

In [6]:
# Create a new DataFrame where each row corresponds to a unique time point, chromosome, start, end, and gene
time_point_rows = []
for idx, row in module_regions_genes.iterrows():
    for time_point, genes in row['Gene'].items():
        for gene in genes:
            time_point_rows.append([row['Chrom'], row['Start'], row['End'], time_point, gene])
            
module_regions_unlabeled = pd.DataFrame(time_point_rows, columns=["Chrom", "Start", "End", "Sample", "Gene"])

In [7]:
# Label Unique Module (Enhancer) Regions
modules_labeled = module_regions_unlabeled
modules_labeled = modules_labeled.drop("Gene", axis = 1)
modules_labeled = modules_labeled.drop("Sample", axis =1)
modules_labeled['Module'] = 'Module_' + modules_labeled.groupby(['Chrom', 'Start', 'End']).ngroup().add(1).astype(str)
modules_labeled = modules_labeled.reindex(['Chrom', 'Start', 'End', 'Module'], axis = 1)
modules_labeled = modules_labeled.drop_duplicates()

In [8]:
# Generate File for Collapsed, Unfiltered Activity-By-Contact Unique Regions

modules_labeled.to_csv("/home/wbd20/Kreimer_Lab/Network/Components/ABC_Network_Unique_Regions_Labeled.tsv",
                       sep = '\t', index = False)

In [9]:
# Generate .bed File for bedtools multicov

module_bed = modules_labeled
module_bed = module_bed.drop('Module', axis = 1)
module_bed.to_csv("/home/wbd20/Kreimer_Lab/Network/Components/ABC_Network_Unfiltered_Regions.bed",
                       sep = '\t', header = None, index = None)

In [10]:
# Create Collapsed, Unfiltered Activity-By-Contact Network (Modules, Genes, Samples)

module_regions_unlabeled = module_regions_unlabeled.merge(modules_labeled, on=["Chrom","Start", 'End'])
module_regions_unlabeled = module_regions_unlabeled.reindex(['Chrom', 'Start', 'End', 'Module', 'Gene', 'Sample'], axis = 1)
module_regions_edges = module_regions_unlabeled

In [11]:
# Generate File for Collapsed, Unfiltered Activity-By-Contact Edges

module_regions_edges.to_csv("/home/wbd20/Kreimer_Lab/Network/Components/ABC_Network_Unfiltered_Edges.tsv",
                                sep = '\t', index = False)

In [12]:
# Execute --> multicov.sh

In [13]:
# Class: ATACSeq & H3K27ac Read Counts, DESeq2 and ImpulseDE2 File Creation 

class ReadCounts: 
    def __init__(self, ngs_method: str, dataframe: pd.DataFrame, replicants: int, timepoint: int, condition: str, batch: str):
        self.ngs_method = ngs_method
        self.dataframe = dataframe
        self.replicants = replicants 
        self.timepoint = timepoint
        self.condition = condition 
        self.batch = batch
        header = ['Module'] + [f'{timepoint}hrs.{ngs_method}_REPLICANT.{i}' for i in range(1, replicants+1)]
        self.dataframe.columns = header
        self.df = self.dataframe

In [14]:
# Function: Replaces Sequence Information with Module Label in ReadCounts Object

def merge_module_labels(file_path, annotation_module_regions: str, output_dir: str):
    annotation_module_regions = pd.read_csv(annotation_module_regions, delimiter="\t")
    for file_name in os.listdir(file_path):
        if file_name.endswith("_PeakCounts.tab"):
            file_to_merge = pd.read_csv(os.path.join(file_path, file_name), delimiter="\t", names=['Chrom', 'Start', 'End', 'Counts_1', 'Counts_2'])
            merged_df = pd.merge(file_to_merge, annotation_module_regions, on=['Chrom', 'Start', 'End'])
            merged_df = merged_df.drop(['Chrom', 'Start', 'End'], axis=1)
            merged_df = merged_df[['Module', 'Counts_1', 'Counts_2']]
            merged_file_name = file_name.split(".")[0] + "_labeled.tab"
            merged_file_path = os.path.join(output_dir, merged_file_name)
            merged_df.to_csv(merged_file_path, sep='\t', index=False, header=None)

In [15]:
# Function: Loads Read Counts into ReadCounts Object 

def read_readcounts(file_paths: list[str], ngs_method: str, replicants: int, timepoints: list[int], conditions: list[str], batches: list[str]):
    readcounts = []
    for i in range(len(file_paths)):
        dataframe = pd.read_csv(file_paths[i], sep = "\t", header = None)
        readcounts.append(ReadCounts(ngs_method[i], dataframe, replicants[i], timepoints[i], conditions[i], batches[i]))
    
    return readcounts

In [16]:
# Function: Generates DESeq2 Condition Files

def generate_deseq2_conditions(rc_list: list[ReadCounts], output_dir: str):
    rc_list = sorted(rc_list, key = lambda x: x.timepoint)
    for i in range(len(rc_list)):
        for j in range(i + 1, len(rc_list)):
            rc1, rc2 = rc_list[i], rc_list[j]
            data = []
            for k in range(1, rc1.replicants + 1):
                data.append([f"{rc1.timepoint}hrs.{rc1.ngs_method}_REPLICANT.{k}", f"{rc1.timepoint}hrs.{rc1.ngs_method}"])
            for k in range(1, rc2.replicants + 1):
                data.append([f"{rc2.timepoint}hrs.{rc2.ngs_method}_REPLICANT.{k}", f"{rc2.timepoint}hrs.{rc2.ngs_method}"])

            df = pd.DataFrame(data, columns = ["", "timepoints"])
            file_name = f"{rc1.timepoint}hrs_{rc2.timepoint}hrs.{rc1.ngs_method}.DESeq2_conditions.csv"
            file_path = os.path.join(output_dir, file_name)
            df.to_csv(file_path, index = False)

In [17]:
# Function: Generates DESeq2 Comparison Files

def generate_deseq2_comparisons(rc_list: list[ReadCounts], output_dir: str):
    rc_list = sorted(rc_list, key = lambda x: x.timepoint)
    for i in range(len(rc_list)):
        for j in range(i + 1, len(rc_list)):
            rc1, rc2 = rc_list[i], rc_list[j]
            df = pd.merge(rc1.df, rc2.df, on=['Module'])
            file_name = f"{rc1.timepoint}hrs_{rc2.timepoint}hrs.{rc1.ngs_method}.DESeq2_comparisons.csv"
            file_path = os.path.join(output_dir, file_name)
            df.to_csv(file_path, index = False)

In [18]:
# Function: Generates ImpulseDE2 Condition Files

def generate_impulsede2_conditions(rc_list: list[ReadCounts], output_dir: str):
    rc_list = sorted(rc_list, key = lambda x: x.timepoint)
    data = []
    timepoints = sorted(list(set([rc.timepoint for rc in rc_list])))
    for rc in rc_list:
        for k in range(1, rc.replicants + 1):
            data.append([f"{rc.timepoint}hrs.{rc.ngs_method}_REPLICANT.{k}", rc.condition, timepoints.index(rc.timepoint) + 1, rc.batch])
    df = pd.DataFrame(data, columns = ["Sample", "Condition", "Time", "Batch"])
    timepoints_str = '_'.join([f"{rc.timepoint}hrs" for rc in rc_list])
    file_name = f"{timepoints_str}_{rc_list[0].ngs_method}.ImpulseDE2_conditions.csv"
    file_path = os.path.join(output_dir, file_name)
    df.to_csv(file_path, index = False)

In [19]:
# Function: Generates ImpulseDE2 Comparison Files

def generate_impulsede2_comparisons(rc_list: list[ReadCounts], output_dir: str):
    rc_list = sorted(rc_list, key = lambda x: x.timepoint)
    df = rc_list[0].df
    for rc in rc_list[1:]:
        df = pd.merge(df, rc.df, on=['Module'], how='inner')
    file_name = f"{'_'.join([f'{rc.timepoint}hrs' for rc in rc_list])}_{rc_list[0].ngs_method}.ImpulseDE2_comparisons.csv"
    file_path = os.path.join(output_dir, file_name)
    df.to_csv(file_path, index = False)

In [20]:
# Create Labeled ATACSeq Read Counts

merge_module_labels('/home/wbd20/Kreimer_Lab/ATACSeq/Read_Counts/Raw/', 
                    '/home/wbd20/Kreimer_Lab/Network/Components/ABC_Network_Unique_Regions_Labeled.tsv',
                    '/home/wbd20/Kreimer_Lab/ATACSeq/Read_Counts/Labeled/')

In [21]:
# Create Labeled H3K27ac Read Counts

merge_module_labels('/home/wbd20/Kreimer_Lab/H3K27ac/Read_Counts/Raw/', 
                    '/home/wbd20/Kreimer_Lab/Network/Components/ABC_Network_Unique_Regions_Labeled.tsv',
                    '/home/wbd20/Kreimer_Lab/H3K27ac/Read_Counts/Labeled/')

In [22]:
# Create ATACSeq ReadCount Objects

file_paths = ['/home/wbd20/Kreimer_Lab/ATACSeq/Read_Counts/Labeled/ATACseq_0hr_enhancers_combined_PeakCounts_labeled.tab', 
              '/home/wbd20/Kreimer_Lab/ATACSeq/Read_Counts/Labeled/ATACseq_3hr_enhancers_combined_PeakCounts_labeled.tab', 
              '/home/wbd20/Kreimer_Lab/ATACSeq/Read_Counts/Labeled/ATACseq_6hr_enhancers_combined_PeakCounts_labeled.tab',
              '/home/wbd20/Kreimer_Lab/ATACSeq/Read_Counts/Labeled/ATACseq_12hr_enhancers_combined_PeakCounts_labeled.tab',
              '/home/wbd20/Kreimer_Lab/ATACSeq/Read_Counts/Labeled/ATACseq_24hr_enhancers_combined_PeakCounts_labeled.tab',
              '/home/wbd20/Kreimer_Lab/ATACSeq/Read_Counts/Labeled/ATACseq_48hr_enhancers_combined_PeakCounts_labeled.tab',
              '/home/wbd20/Kreimer_Lab/ATACSeq/Read_Counts/Labeled/ATACseq_72hr_enhancers_combined_PeakCounts_labeled.tab',
              ]

ngs_method = ["ATACSeq", "ATACSeq", "ATACSeq", "ATACSeq", "ATACSeq", "ATACSeq", "ATACSeq"]
replicants = [2, 2, 2, 2, 2, 2, 2]
timepoints = [0, 3, 6, 12, 24, 48, 72]
condition = ["case", "case", "case", "case", "case", "case", "case"]
batch = ["B_NULL", "B_NULL", "B_NULL", "B_NULL", "B_NULL", "B_NULL", "B_NULL"]

ATACSeq_ReadCounts = read_readcounts(file_paths, ngs_method, replicants, timepoints, condition, batch)

In [23]:
# Create H3K27ac ReadCount Objects

file_paths = ['/home/wbd20/Kreimer_Lab/H3K27ac/Read_Counts/Labeled/H3K27ac_0hr_enhancers_combined_PeakCounts_labeled.tab', 
              '/home/wbd20/Kreimer_Lab/H3K27ac/Read_Counts/Labeled/H3K27ac_3hr_enhancers_combined_PeakCounts_labeled.tab', 
              '/home/wbd20/Kreimer_Lab/H3K27ac/Read_Counts/Labeled/H3K27ac_6hr_enhancers_combined_PeakCounts_labeled.tab',
              '/home/wbd20/Kreimer_Lab/H3K27ac/Read_Counts/Labeled/H3K27ac_12hr_enhancers_combined_PeakCounts_labeled.tab',
              '/home/wbd20/Kreimer_Lab/H3K27ac/Read_Counts/Labeled/H3K27ac_24hr_enhancers_combined_PeakCounts_labeled.tab',
              '/home/wbd20/Kreimer_Lab/H3K27ac/Read_Counts/Labeled/H3K27ac_48hr_enhancers_combined_PeakCounts_labeled.tab',
              '/home/wbd20/Kreimer_Lab/H3K27ac/Read_Counts/Labeled/H3K27ac_72hr_enhancers_combined_PeakCounts_labeled.tab',
              ]

ngs_method = ["H3K27ac", "H3K27ac", "H3K27ac", "H3K27ac", "H3K27ac", "H3K27ac", "H3K27ac"]
replicants = [2, 2, 2, 2, 2, 2, 2]
timepoints = [0, 3, 6, 12, 24, 48, 72]
condition = ["case", "case", "case", "case", "case", "case", "case"]
batch = ["B_NULL", "B_NULL", "B_NULL", "B_NULL", "B_NULL", "B_NULL", "B_NULL"]

H3K27ac_ReadCounts = read_readcounts(file_paths, ngs_method, replicants, timepoints, condition, batch)

In [24]:
# Generate ATACSeq DESeq2 Files

generate_deseq2_conditions(ATACSeq_ReadCounts,'/home/wbd20/Kreimer_Lab/ATACSeq/DESeq2/Conditions/')
generate_deseq2_comparisons(ATACSeq_ReadCounts,'/home/wbd20/Kreimer_Lab/ATACSeq/DESeq2/Comparisons/')

In [25]:
# Generate H3K27ac DESeq2 Files

generate_deseq2_conditions(H3K27ac_ReadCounts,'/home/wbd20/Kreimer_Lab/H3K27ac/DESeq2/Conditions/')
generate_deseq2_comparisons(H3K27ac_ReadCounts,'/home/wbd20/Kreimer_Lab/H3K27ac/DESeq2/Comparisons/')

In [26]:
# Generate ATACSeq ImpulseDE2 Files

generate_impulsede2_conditions(ATACSeq_ReadCounts,'/home/wbd20/Kreimer_Lab/ATACSeq/ImpulseDE2/Conditions/')
generate_impulsede2_comparisons(ATACSeq_ReadCounts,'/home/wbd20/Kreimer_Lab/ATACSeq/ImpulseDE2/Comparisons/')

In [27]:
# Generate H3K27ac ImpulseDE2 Files

generate_impulsede2_conditions(H3K27ac_ReadCounts,'/home/wbd20/Kreimer_Lab/H3K27ac/ImpulseDE2/Conditions/')
generate_impulsede2_comparisons(H3K27ac_ReadCounts,'/home/wbd20/Kreimer_Lab/H3K27ac/ImpulseDE2/Comparisons/')

In [28]:
# Class: RNASeq Read Counts, DESeq2 and ImpulseDE2 File Creation 

class ReadCounts: 
    def __init__(self, ngs_method: str, dataframe: pd.DataFrame, replicants: int, timepoint: int, condition: str, batch: str):
        self.ngs_method = ngs_method
        self.dataframe = dataframe
        self.replicants = replicants 
        self.timepoint = timepoint
        self.condition = condition 
        self.batch = batch
        header = ['Gene'] + [f'{timepoint}hrs.{ngs_method}_REPLICANT.{i}' for i in range(1, replicants+1)]
        self.dataframe.columns = header
        self.df = self.dataframe

In [29]:
# Function: Generates DESeq2 Comparison Files (RNASeq)

def generate_deseq2_comparisons(rc_list: list[ReadCounts], output_dir: str):
    rc_list = sorted(rc_list, key = lambda x: x.timepoint)
    for i in range(len(rc_list)):
        for j in range(i + 1, len(rc_list)):
            rc1, rc2 = rc_list[i], rc_list[j]
            df = pd.merge(rc1.df, rc2.df, on=['Gene'])
            file_name = f"{rc1.timepoint}hrs_{rc2.timepoint}hrs.{rc1.ngs_method}.DESeq2_comparisons.csv"
            file_path = os.path.join(output_dir, file_name)
            df.to_csv(file_path, index = False)

In [30]:
# Function: Generates ImpulseDE2 Comparison Files (RNASeq)

def generate_impulsede2_comparisons(rc_list: list[ReadCounts], output_dir: str):
    rc_list = sorted(rc_list, key = lambda x: x.timepoint)
    df = rc_list[0].df
    for rc in rc_list[1:]:
        df = pd.merge(df, rc.df, on=['Gene'], how='inner')
    file_name = f"{'_'.join([f'{rc.timepoint}hrs' for rc in rc_list])}_{rc_list[0].ngs_method}.ImpulseDE2_comparisons.csv"
    file_path = os.path.join(output_dir, file_name)
    df.to_csv(file_path, index = False)

In [31]:
# Load RNASeq Data
Neuro_7TP_RNASeq_Genes = pd.read_csv('/projectsp/f_ak1833_1/Neuro_7TP_RNAseq_OUT/Neuro_7TP_RNASeq_Genes.tsv', sep = '\t')
Neuro_7TP_RNASeq_Genes = Neuro_7TP_RNASeq_Genes.drop('Gene', axis = 1)
Neuro_7TP_RNASeq_Genes = Neuro_7TP_RNASeq_Genes.rename(columns={'Symbol': 'Gene'})

# Remove Unnecessary FPKM, TPM Values 
cols_FPKM = Neuro_7TP_RNASeq_Genes.columns[Neuro_7TP_RNASeq_Genes.columns.str.endswith("_FPKM")]
cols_TPM = Neuro_7TP_RNASeq_Genes.columns[Neuro_7TP_RNASeq_Genes.columns.str.endswith("_TPM")] 
Neuro_7TP_RNASeq_Genes = Neuro_7TP_RNASeq_Genes.drop(cols_FPKM, axis = 1)
Neuro_7TP_RNASeq_Genes = Neuro_7TP_RNASeq_Genes.drop(cols_TPM, axis = 1)

# Remove NaN Values from Read Counts
Neuro_7TP_RNASeq_Genes = Neuro_7TP_RNASeq_Genes.dropna()

# Convert Read Counts Floats to Ints
Neuro_7TP_RNASeq_Genes = Neuro_7TP_RNASeq_Genes.round(0)
numeric_cols = Neuro_7TP_RNASeq_Genes.select_dtypes(include=['float']).columns
Neuro_7TP_RNASeq_Genes[numeric_cols] = Neuro_7TP_RNASeq_Genes[numeric_cols].astype(int)

# Assign Column Names
cols = ['Gene',
        '0hrs.RNASeq_REPLICANT.1',
        '0hrs.RNASeq_REPLICANT.2',
        '0hrs.RNASeq_REPLICANT.3',
        '3hrs.RNASeq_REPLICANT.1',
        '3hrs.RNASeq_REPLICANT.2',
        '3hrs.RNASeq_REPLICANT.3',
        '6hrs.RNASeq_REPLICANT.1',
        '6hrs.RNASeq_REPLICANT.2',
        '6hrs.RNASeq_REPLICANT.3',
        '12hrs.RNASeq_REPLICANT.1',
        '12hrs.RNASeq_REPLICANT.2',
        '12hrs.RNASeq_REPLICANT.3',
        '24hrs.RNASeq_REPLICANT.1',
        '24hrs.RNASeq_REPLICANT.2',
        '24hrs.RNASeq_REPLICANT.3',
        '48hrs.RNASeq_REPLICANT.1',
        '48hrs.RNASeq_REPLICANT.2',
        '48hrs.RNASeq_REPLICANT.3',
        '72hrs.RNASeq_REPLICANT.1',
        '72hrs.RNASeq_REPLICANT.2',
        '72hrs.RNASeq_REPLICANT.3'
       ]

Neuro_7TP_RNASeq_Genes.columns = cols

# Create Read Counts per Time Point

Neuro_7TP_RNASeq_Genes_0hrs = Neuro_7TP_RNASeq_Genes[['Gene', 
                                                      '0hrs.RNASeq_REPLICANT.1', 
                                                      '0hrs.RNASeq_REPLICANT.2', 
                                                      '0hrs.RNASeq_REPLICANT.3']]

Neuro_7TP_RNASeq_Genes_0hrs.to_csv("/home/wbd20/Kreimer_Lab/RNASeq/Read_Counts/Labeled/RNAseq_0hr_genes_combined_PeakCounts_labeled.tab",
                                   sep = '\t', index = False, header = None)

Neuro_7TP_RNASeq_Genes_3hrs = Neuro_7TP_RNASeq_Genes[['Gene', 
                                                      '3hrs.RNASeq_REPLICANT.1', 
                                                      '3hrs.RNASeq_REPLICANT.2', 
                                                      '3hrs.RNASeq_REPLICANT.3']]

Neuro_7TP_RNASeq_Genes_3hrs.to_csv("/home/wbd20/Kreimer_Lab/RNASeq/Read_Counts/Labeled/RNAseq_3hr_genes_combined_PeakCounts_labeled.tab",
                                   sep = '\t' , index = False, header = None)


Neuro_7TP_RNASeq_Genes_6hrs = Neuro_7TP_RNASeq_Genes[['Gene', 
                                                      '6hrs.RNASeq_REPLICANT.1', 
                                                      '6hrs.RNASeq_REPLICANT.2', 
                                                      '6hrs.RNASeq_REPLICANT.3']]

Neuro_7TP_RNASeq_Genes_6hrs.to_csv("/home/wbd20/Kreimer_Lab/RNASeq/Read_Counts/Labeled/RNAseq_6hr_genes_combined_PeakCounts_labeled.tab",
                                   sep = '\t' , index = False, header = None)


Neuro_7TP_RNASeq_Genes_12hrs = Neuro_7TP_RNASeq_Genes[['Gene', 
                                                      '12hrs.RNASeq_REPLICANT.1', 
                                                      '12hrs.RNASeq_REPLICANT.2', 
                                                      '12hrs.RNASeq_REPLICANT.3']]

Neuro_7TP_RNASeq_Genes_12hrs.to_csv("/home/wbd20/Kreimer_Lab/RNASeq/Read_Counts/Labeled/RNAseq_12hr_genes_combined_PeakCounts_labeled.tab",
                                   sep = '\t' , index = False, header = None)

Neuro_7TP_RNASeq_Genes_24hrs = Neuro_7TP_RNASeq_Genes[['Gene', 
                                                      '24hrs.RNASeq_REPLICANT.1', 
                                                      '24hrs.RNASeq_REPLICANT.2', 
                                                      '24hrs.RNASeq_REPLICANT.3']]

Neuro_7TP_RNASeq_Genes_24hrs.to_csv("/home/wbd20/Kreimer_Lab/RNASeq/Read_Counts/Labeled/RNAseq_24hr_genes_combined_PeakCounts_labeled.tab",
                                   sep = '\t' , index = False, header = None)

Neuro_7TP_RNASeq_Genes_48hrs = Neuro_7TP_RNASeq_Genes[['Gene', 
                                                      '48hrs.RNASeq_REPLICANT.1', 
                                                      '48hrs.RNASeq_REPLICANT.2', 
                                                      '48hrs.RNASeq_REPLICANT.3']]

Neuro_7TP_RNASeq_Genes_48hrs.to_csv("/home/wbd20/Kreimer_Lab/RNASeq/Read_Counts/Labeled/RNAseq_48hr_genes_combined_PeakCounts_labeled.tab",
                                   sep = '\t' , index = False, header = None)

Neuro_7TP_RNASeq_Genes_72hrs = Neuro_7TP_RNASeq_Genes[['Gene', 
                                                      '72hrs.RNASeq_REPLICANT.1', 
                                                      '72hrs.RNASeq_REPLICANT.2', 
                                                      '72hrs.RNASeq_REPLICANT.3']]

Neuro_7TP_RNASeq_Genes_72hrs.to_csv("/home/wbd20/Kreimer_Lab/RNASeq/Read_Counts/Labeled/RNAseq_72hr_genes_combined_PeakCounts_labeled.tab",
                                   sep = '\t' , index = False, header = None)

In [32]:
# Create H3K27ac ReadCount Objects

file_paths = ['/home/wbd20/Kreimer_Lab/RNASeq/Read_Counts/Labeled/RNAseq_0hr_genes_combined_PeakCounts_labeled.tab', 
              '/home/wbd20/Kreimer_Lab/RNASeq/Read_Counts/Labeled/RNAseq_3hr_genes_combined_PeakCounts_labeled.tab', 
              '/home/wbd20/Kreimer_Lab/RNASeq/Read_Counts/Labeled/RNAseq_6hr_genes_combined_PeakCounts_labeled.tab',
              '/home/wbd20/Kreimer_Lab/RNASeq/Read_Counts/Labeled/RNAseq_12hr_genes_combined_PeakCounts_labeled.tab',
              '/home/wbd20/Kreimer_Lab/RNASeq/Read_Counts/Labeled/RNAseq_24hr_genes_combined_PeakCounts_labeled.tab',
              '/home/wbd20/Kreimer_Lab/RNASeq/Read_Counts/Labeled/RNAseq_48hr_genes_combined_PeakCounts_labeled.tab',
              '/home/wbd20/Kreimer_Lab/RNASeq/Read_Counts/Labeled/RNAseq_72hr_genes_combined_PeakCounts_labeled.tab',
              ]

ngs_method = ["RNASeq", "RNASeq", "RNASeq", "RNASeq", "RNASeq", "RNASeq", "RNASeq"]
replicants = [3, 3, 3, 3, 3, 3, 3]
timepoints = [0, 3, 6, 12, 24, 48, 72]
condition = ["case", "case", "case", "case", "case", "case", "case"]
batch = ["B_NULL", "B_NULL", "B_NULL", "B_NULL", "B_NULL", "B_NULL", "B_NULL"]

RNASeq_ReadCounts = read_readcounts(file_paths, ngs_method, replicants, timepoints, condition, batch)

In [33]:
# Generate RNASeq DESeq2 Files

generate_deseq2_conditions(RNASeq_ReadCounts,'/home/wbd20/Kreimer_Lab/RNASeq/DESeq2/Conditions/')
generate_deseq2_comparisons(RNASeq_ReadCounts,'/home/wbd20/Kreimer_Lab/RNASeq/DESeq2/Comparisons/')

In [34]:
# Generate RNASeq ImpulseDE2 Files

generate_impulsede2_conditions(RNASeq_ReadCounts,'/home/wbd20/Kreimer_Lab/RNASeq/ImpulseDE2/Conditions/')
generate_impulsede2_comparisons(RNASeq_ReadCounts,'/home/wbd20/Kreimer_Lab/RNASeq/ImpulseDE2/Comparisons/')

In [35]:
# Execute --> de.sh

In [36]:
# Function: Compiles DESeq2 Results into File (1)

def collapse_deseq2_results(base_dir: str, output_file: str):
    dfs = []
    for file_name in filter(lambda f: os.path.splitext(f)[1] == '.csv', next(os.walk(base_dir))[2]):
        df = pd.read_csv(os.path.join(base_dir, file_name), sep=',', header=None)
        df['file_name'] = file_name
        dfs.append(df)
    pd.concat(dfs, axis = 0).to_csv(output_file, sep=',', header=False, index=False)

In [37]:
# Function: Extracts Modules (Enhancers) or Genes from DESeq2 Files

def polish_deseq2_results(file_name: str, col_name: str, output_file: str):
    df = pd.read_csv(file_name)
    df.columns = [col_name, "baseMean", "log2FoldChange", "lfcSE", "stat","pvalue","padj","Timepoint"]
    df = df.drop("baseMean", axis = 'columns')
    df = df.drop("log2FoldChange", axis = 'columns')
    df = df.drop("lfcSE", axis = 'columns')
    df = df.drop("stat", axis = 'columns')
    df = df.drop("pvalue", axis = 'columns')
    df = df.drop("padj", axis = 'columns')
    df = df.drop("Timepoint", axis = 'columns')
    df = df.drop_duplicates()
    df.to_csv(output_file, index=False, sep = '\t')

In [38]:
# Function: Extracts Modules (Enhancers) or Genes from DESeq2 Files

def polish_impulsede2_results(file_name: str, col_name: str, output_file: str):
    df = pd.read_csv(file_name)
    df.columns = ["", col_name, "p", "padj", "loglik_full", "loglik_red", "df_full", "df_red", "mean", "converge_impulse", "converge_const", "allZero"]
    df = df.drop("p", axis = 'columns')
    df = df.drop("padj", axis = 'columns')
    df = df.drop("loglik_full", axis = 'columns')
    df = df.drop("loglik_red", axis = 'columns')
    df = df.drop("df_full", axis = 'columns')
    df = df.drop("df_red", axis = 'columns')
    df = df.drop("mean", axis = 'columns')
    df = df.drop("converge_impulse", axis = 'columns')
    df = df.drop("converge_const", axis = 'columns')
    df = df.drop("allZero", axis = 'columns')
    df = df.drop("", axis = 'columns')
    df = df.drop_duplicates()

    df.to_csv(output_file, index=False, sep = '\t')

In [39]:
# Collapse ATACSeq DESeq2 Results

collapse_deseq2_results('/home/wbd20/Kreimer_Lab/ATACSeq/DESeq2/Results/', 
                        '/home/wbd20/Kreimer_Lab/ATACSeq/DESeq2/Results/Collapsed/ATACSeq.DESeq2_collapsed.csv')

In [40]:
# Collapse H3K27ac DESeq2 Results

collapse_deseq2_results('/home/wbd20/Kreimer_Lab/H3K27ac/DESeq2/Results/', 
                        '/home/wbd20/Kreimer_Lab/H3K27ac/DESeq2/Results/Collapsed/H3K27ac.DESeq2_collapsed.csv')

In [41]:
# Collapse RNASeq DESeq2 Results

collapse_deseq2_results('/home/wbd20/Kreimer_Lab/RNASeq/DESeq2/Results/', 
                        '/home/wbd20/Kreimer_Lab/RNASeq/DESeq2/Results/Collapsed/RNASeq.DESeq2_collapsed.csv')

In [42]:
# Polish ATACSeq DESeq2 Results

polish_deseq2_results('/home/wbd20/Kreimer_Lab/ATACSeq/DESeq2/Results/Collapsed/ATACSeq.DESeq2_collapsed.csv',
                      'Module',
                      "/home/wbd20/Kreimer_Lab/Network/Components/ATACSeq.DESeq2_regions.txt")

In [43]:
# Polish H3K27ac DESeq2 Results

polish_deseq2_results('/home/wbd20/Kreimer_Lab/H3K27ac/DESeq2/Results/Collapsed/H3K27ac.DESeq2_collapsed.csv',
                      'Module',
                      '/home/wbd20/Kreimer_Lab/Network/Components/H3K27ac.DESeq2_regions.txt')

In [44]:
# Polish RNASeq DESeq2 Results

polish_deseq2_results('/home/wbd20/Kreimer_Lab/RNASeq/DESeq2/Results/Collapsed/RNASeq.DESeq2_collapsed.csv',
                      'Gene',
                      '/home/wbd20/Kreimer_Lab/Network/Components/RNASeq.DESeq2_genes.txt')

In [45]:
# Polish ATACSeq ImpulseDE2 Results

polish_impulsede2_results("/home/wbd20/Kreimer_Lab/ATACSeq/ImpulseDE2/Results/ATACSeq.ImpulseDE2_results.csv", 
                          "Module", 
                          "/home/wbd20/Kreimer_Lab/Network/Components/ATACSeq.ImpulseDE2_regions.txt")

In [46]:
# Polish H3K27ac ImpulseDE2 Results

polish_impulsede2_results("/home/wbd20/Kreimer_Lab/H3K27ac/ImpulseDE2/Results/H3K27ac.ImpulseDE2_results.csv", 
                          "Module", 
                          "/home/wbd20/Kreimer_Lab/Network/Components/H3K27ac.ImpulseDE2_regions.txt")

In [47]:
# Polish RNASeq ImpulseDE2 Results

polish_impulsede2_results("/home/wbd20/Kreimer_Lab/RNASeq/ImpulseDE2/Results/RNASeq.ImpulseDE2_results.csv", 
                          "Gene", 
                          "/home/wbd20/Kreimer_Lab/Network/Components/RNASeq.ImpulseDE2_genes.txt")

In [48]:
# Function: Quantile Filtration for Read Counts

def rc_quantile_filter(file_name: str, output_file: str, quantile: float):
    df = pd.read_csv(file_name)
    cols_numeric = df.select_dtypes(include = 'number').columns
    df['Average'] = df[cols_numeric].mean(axis = 1)
    df['Maximum'] = df[cols_numeric].max(axis = 1)
    df = df[['Module', 'Average', 'Maximum']]
    avg_threshold = df['Average'].quantile(quantile)
    max_threshold = df['Maximum'].quantile(quantile)
    quantile = df[(df['Average'] >= avg_threshold) & (df['Maximum'] >= max_threshold)]
    quantile = quantile.drop('Average', axis = 1)
    quantile = quantile.drop('Maximum', axis = 1)
    quantile.to_csv(output_file, index=False, sep ='\t')

In [49]:
# Grab Top 10% Modules (Enhancers), ATACSeq Read Count

rc_quantile_filter('/home/wbd20/Kreimer_Lab/ATACSeq/ImpulseDE2/Comparisons/0hrs_3hrs_6hrs_12hrs_24hrs_48hrs_72hrs_ATACSeq.ImpulseDE2_comparisons.csv',
                   "/home/wbd20/Kreimer_Lab/Network/Components/ATACSeq.Quantile_regions.txt", 0.95)

In [50]:
# Grab Top 10% Modules (Enhancers), H3K27ac Read Count

rc_quantile_filter('/home/wbd20/Kreimer_Lab/H3K27ac/ImpulseDE2/Comparisons/0hrs_3hrs_6hrs_12hrs_24hrs_48hrs_72hrs_H3K27ac.ImpulseDE2_comparisons.csv',
                   "/home/wbd20/Kreimer_Lab/Network/Components/H3K27ac.Quantile_regions.txt", 0.95)

In [51]:
# Load RNASeq Data
Neuro_7TP_RNASeq_Genes = pd.read_csv('/projectsp/f_ak1833_1/Neuro_7TP_RNAseq_OUT/Neuro_7TP_RNASeq_Genes.tsv', sep = '\t')
Neuro_7TP_RNASeq_Genes = Neuro_7TP_RNASeq_Genes.drop('Gene', axis = 1)
Neuro_7TP_RNASeq_Genes = Neuro_7TP_RNASeq_Genes.rename(columns={'Symbol': 'Gene'})

# Remove Unnecessary FPKM, countM Values 
cols_FPKM = Neuro_7TP_RNASeq_Genes.columns[Neuro_7TP_RNASeq_Genes.columns.str.endswith("_FPKM")]
cols_countM = Neuro_7TP_RNASeq_Genes.columns[Neuro_7TP_RNASeq_Genes.columns.str.endswith("_countM")] 
Neuro_7TP_RNASeq_Genes = Neuro_7TP_RNASeq_Genes.drop(cols_FPKM, axis = 1)
Neuro_7TP_RNASeq_Genes = Neuro_7TP_RNASeq_Genes.drop(cols_countM, axis = 1)

# Remove NaN Values from Read Counts
Neuro_7TP_RNASeq_Genes = Neuro_7TP_RNASeq_Genes.dropna()

# Convert Read Counts Floats to Ints
Neuro_7TP_RNASeq_Genes = Neuro_7TP_RNASeq_Genes.round(0)
numeric_cols = Neuro_7TP_RNASeq_Genes.select_dtypes(include=['float']).columns
Neuro_7TP_RNASeq_Genes[numeric_cols] = Neuro_7TP_RNASeq_Genes[numeric_cols].astype(int)

# Quantile Filter for RNASeq TPM
cols_numeric = Neuro_7TP_RNASeq_Genes.select_dtypes(include = 'number').columns
Neuro_7TP_RNASeq_Genes['Average'] = Neuro_7TP_RNASeq_Genes[cols_numeric].mean(axis = 1)
Neuro_7TP_RNASeq_Genes['Maximum'] = Neuro_7TP_RNASeq_Genes[cols_numeric].max(axis = 1)
Neuro_7TP_RNASeq_Genes = Neuro_7TP_RNASeq_Genes[['Gene', 'Average', 'Maximum']]
avg_threshold = Neuro_7TP_RNASeq_Genes['Average'].quantile(0.95)
max_threshold = Neuro_7TP_RNASeq_Genes['Maximum'].quantile(0.95)
quantile = Neuro_7TP_RNASeq_Genes[(Neuro_7TP_RNASeq_Genes['Average'] >= avg_threshold) & (Neuro_7TP_RNASeq_Genes['Maximum'] >= max_threshold)]
quantile = quantile.drop('Average', axis = 1)
quantile = quantile.drop('Maximum', axis = 1)
quantile.to_csv("/home/wbd20/Kreimer_Lab/Network/Components/RNASeq.Quantile_genes.txt", index=False, sep = '\t')

In [52]:
# Load Unfiltered Activity-By-Contact Model Network
NW_Unfiltered = pd.read_csv('/home/wbd20/Kreimer_Lab/Network/Components/ABC_Network_Unfiltered_Edges.tsv', sep='\t')

# Load Modules (Enhancers) DESeq2, ImpulseDE2 Results
ATACSeq_DESeq2 = pd.read_csv('/home/wbd20/Kreimer_Lab/Network/Components/ATACSeq.DESeq2_regions.txt', sep='\t')
ATACSeq_ImpulseDE2 = pd.read_csv('/home/wbd20/Kreimer_Lab/Network/Components/ATACSeq.ImpulseDE2_regions.txt', sep='\t')
H3K27ac_DESeq2 = pd.read_csv('/home/wbd20/Kreimer_Lab/Network/Components/H3K27ac.DESeq2_regions.txt', sep='\t')
H3K27ac_ImpulseDE2 = pd.read_csv('/home/wbd20/Kreimer_Lab/Network/Components/H3K27ac.ImpulseDE2_regions.txt', sep='\t')

# Load Modules (Enhancers) Quantile Results
ATACSeq_Quantile = pd.read_csv('/home/wbd20/Kreimer_Lab/Network/Components/ATACSeq.Quantile_regions.txt', sep='\t')
H3K27ac_Quantile = pd.read_csv('/home/wbd20/Kreimer_Lab/Network/Components/H3K27ac.Quantile_regions.txt', sep='\t')

# Load Genes DESeq2, ImpulseDE2 Results
RNASeq_DESeq2 = pd.read_csv('/home/wbd20/Kreimer_Lab/Network/Components/RNASeq.DESeq2_genes.txt', sep='\t')
RNASeq_ImpulseDE2 = pd.read_csv('/home/wbd20/Kreimer_Lab/Network/Components/RNASeq.ImpulseDE2_genes.txt', sep='\t')

# Load Genes Quantile Results
Quantile_Genes = pd.read_csv('/home/wbd20/Kreimer_Lab/Network/Components/RNASeq.Quantile_genes.txt', sep='\t')

# Generate Network-Included Modules (Enhancers)
DESeq2 = pd.merge(ATACSeq_DESeq2, H3K27ac_DESeq2, on = 'Module')
ImpulseDE2 = pd.merge(ATACSeq_ImpulseDE2, H3K27ac_ImpulseDE2, on = 'Module')
DE_Modules = pd.merge(DESeq2, ImpulseDE2, on = 'Module')
Quantile_Modules =  pd.merge(ATACSeq_Quantile, H3K27ac_Quantile, on = 'Module')
NW_Modules = pd.concat([DE_Modules, Quantile_Modules])

# Generate Network-Included Genes
DE_Genes = pd.merge(RNASeq_DESeq2, RNASeq_ImpulseDE2, on = 'Gene')
NW_Genes = pd.concat([DE_Genes, Quantile_Genes])

# Intersect Network-Included Modules (Enhancers), Network-Included Genes with Unfiltered Regions
NW_Unfiltered = pd.merge(NW_Unfiltered, NW_Modules, on = ['Module'])
NW_Unfiltered = pd.merge(NW_Unfiltered, NW_Genes, on = ['Gene'])
NW_Unfiltered = NW_Unfiltered[['Chrom','Start','End','Module','Gene','Sample']]
NW_Unfiltered = NW_Unfiltered.drop_duplicates()

# Save Filtered Activity-By-Contact Network
NW_Filtered = NW_Unfiltered
NW_Filtered.to_csv('/home/wbd20/Kreimer_Lab/Network/ABC_Network_Filtered_Edges.tsv', sep ='\t', index = None)

# Generate Filtered Activity-By-Contact Network .bed File

NW_bed = pd.read_csv('/home/wbd20/Kreimer_Lab/Network/ABC_Network_Filtered_Edges.tsv', sep ='\t')
NW_bed = NW_bed.drop('Sample', axis = 1)
NW_bed = NW_bed.drop('Module', axis = 1)
NW_bed = NW_bed.drop('Gene', axis = 1)
NW_bed = NW_bed.drop_duplicates()
NW_bed.to_csv('/home/wbd20/Kreimer_Lab/Network/ABC_Network_Filtered_Regions.bed', sep ='\t', index = None, header = None)

In [53]:
# Generates Filtered Activity-By-Contact Networks for Each Individual Timepoint

NW_Filtered = pd.read_csv('/home/wbd20/Kreimer_Lab/Network/ABC_Network_Filtered_Edges.tsv', sep='\t')

for sample in NW_Filtered['Sample'].unique():
    
    sample_df = NW_Filtered[NW_Filtered['Sample'] == sample]
    sample_df = sample_df.drop('Sample', axis=1)
    
    file_name = f'/home/wbd20/Kreimer_Lab/Network/Timepoints/Tables/{sample}.ABC_Network_Filtered_Edges.tsv'
    sample_df.to_csv(file_name, sep='\t', index=False)

In [54]:
# Generates .bed for Each Individual Timepoint

NW_Filtered = pd.read_csv('/home/wbd20/Kreimer_Lab/Network/ABC_Network_Filtered_Edges.tsv', sep='\t')

for sample in NW_Filtered['Sample'].unique():
    
    sample_df = NW_Filtered[NW_Filtered['Sample'] == sample]
    sample_df = sample_df.drop('Module', axis=1)
    sample_df = sample_df.drop('Gene', axis=1)
    sample_df = sample_df.drop('Sample', axis=1)
    
    file_name = f'/home/wbd20/Kreimer_Lab/Network/Timepoints/BED/{sample}.ABC_Network_Filtered_Regions.bed'
    sample_df.to_csv(file_name, sep='\t', index=False, header = None)

In [55]:
# Report Network Statistics

average_module_length = NW_Filtered['End'] - NW_Filtered['Start']
average_module_length = average_module_length.mean()

unique_modules = NW_Filtered['Module'].nunique()

unique_genes = NW_Filtered['Gene'].nunique()

gene_module_ratio = unique_genes / unique_modules

sample_counts = NW_Filtered['Sample'].value_counts()

sample_percentages = NW_Filtered['Sample'].value_counts(normalize = True) * 100

print("Average length of Module:", average_module_length)
print("Unique Modules:", unique_modules)
print("Unique Genes:", unique_genes)
print("Gene to Modules ratio:", gene_module_ratio)
print("Counts per sample:\n", sample_counts)
print("Counts per sample as a percent:\n", sample_percentages)

Average length of Module: 1047.827392578125
Unique Modules: 1628
Unique Genes: 3133
Gene to Modules ratio: 1.9244471744471745
Counts per sample:
 24h_sample    2569
3h_sample     2469
6h_sample     2362
0h_sample     2327
12h_sample    2281
72h_sample    2242
48h_sample    2134
Name: Sample, dtype: int64
Counts per sample as a percent:
 24h_sample    15.679932
3h_sample     15.069580
6h_sample     14.416504
0h_sample     14.202881
12h_sample    13.922119
72h_sample    13.684082
48h_sample    13.024902
Name: Sample, dtype: float64
